In [1]:
import re
import pymorphy2
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

In [2]:
tags = ('A', 'ADV', 'ADVPRO', 'ANUM', 'APRO', 'COM', 'CONJ', 'INTJ', 'NUM', 'PART', 'PR', 'S', 'SPRO', 'V', 'PRT', 'GRND')

In [3]:
def standardize(tag):
    new_tags = {'ADV': 'ADVB', 'ADVPRO': 'ADVB', 'ANUM': 'NUMR', 'APRO': 'ADJ', 'NUM': 'NUMR',
                'PART': 'PRCL', 'PR': 'PREP', 'S': 'NOUN', 'SPRO': 'NPRO', 'V': 'VERB', 'A': 'ADJ'}
    if tag in new_tags:
        new_tag = new_tags[tag]
        return new_tag
    return tag

In [4]:
def lemmatization(word, tag=None):
    w = morph.parse(word)
    l = w[0].normal_form
    if tag:
        tag = standardize(tag)
        for ana in w:
            if tag == 'ADJ' or tag == 'PRT':
                if ana.tag.POS.startswith(tag):
                    l = ana.normal_form
                    break
            else:
                if ana.tag.POS == tag:
                    l = ana.normal_form
                    break
    return (l)

In [5]:
def only_one_word(phrase):
    to_search = {'token': None, 'lemma': None, 'pos': None}
    if phrase[0].upper() in tags:
        phrase = phrase[0].upper()
        to_search['pos'] = phrase
        #print('ищем в столбце pos')
    else:
        phrase = phrase[0].lower()
        if re.search(r'[a-z]', phrase):
            print('Латинские символы могут быть использованы только для частеречных тегов')
            return None
        elif phrase[0] == '"':
            to_search['token'] = phrase[1:-1]
            #print('ищем в столбце token')
        else:
            l = lemmatization(phrase)
            to_search['lemma'] = l
    return to_search

In [6]:
def token_and_tag(phrase):
    if re.search(r'[a-z]', phrase[0].lower()):
        print('Формат введенного запроса неверный')
        return None
    else:
        to_search = {'token': None, 'lemma': None, 'pos': None}
        w = phrase[0].lower() #слово
        tag = phrase[1] #тег
        if w[0] == '"':
            to_search['token'] = w[1:-1]
        else:
            l = lemmatization(w, tag)
            to_search['lemma'] = l
        p = phrase[1].upper()
        if p not in tags:
            print('Указан неправильный частеречный тег')
            return None
        else:
            to_search['pos'] = p
            return to_search

In [7]:
def main_search(q):
    if re.search(r'[^a-zA-Zа-яА-Я +"]', q):
        print('В запросе присутствуют некорректные символы')
        return None
    else:
        request = []
        q = q.split()
        if len(q) > 3:
            print('Превышена длина запроса')
        elif len(q) == 1:    # значит в запросе одно слово
            q1 = q[0].split('+')
            if len(q1) == len(q):    # либо только токен, либо только тег
                result = only_one_word(q1)
                if result is not None:
                    request.append(result)
            else:    # в запросе токен и тег
                result = token_and_tag(q1)
                if result is not None:
                    request.append(result)
        else:    # в запросе не одно слово
            for element in q:
                q1 = element.split('+')
                if len(q1) == 1:    # либо только токен, либо только тег
                    result = only_one_word(q1)
                    if result is not None:
                        request.append(result)
                    else:
                        request = []
                        break
                else:    # в запросе токен и тег
                    result = token_and_tag(q1)
                    if result is not None:
                        request.append(result)
                    else:
                        request = []
                        break
    return request

In [8]:
q = 'мыла+V'    # сюда введите запрос
q_list = main_search(q)

In [9]:
print(q_list)

[{'token': None, 'lemma': 'мыть', 'pos': 'V'}]


In [10]:
import sqlite3

In [11]:
conn = sqlite3.connect('poems_corpus.db')
cur = conn.cursor()

In [12]:
def token_and_pos(dct):
    query = """
    SELECT token_id, sent_id
    FROM tokens
    WHERE token = ? AND pos = ?
    """
    cur.execute(query, (dct['token'], dct['pos']))
    result = cur.fetchall()
    return result

In [13]:
def lemma_and_pos(dct):
    query = """
    SELECT token_id, sent_id
    FROM tokens
    WHERE lemma = ? AND pos = ?
    """
    cur.execute(query, (dct['lemma'], dct['pos']))
    result = cur.fetchall()
    return result

In [14]:
def only_token(dct):
    query = """
    SELECT token_id, sent_id
    FROM tokens
    WHERE token = ?
    """
    cur.execute(query, (dct['token'],))
    result = cur.fetchall()
    return result

In [15]:
def only_lemma(dct):
    query = """
    SELECT token_id, sent_id
    FROM tokens
    WHERE lemma = ?
    """
    cur.execute(query, (dct['lemma'],))
    result = cur.fetchall()
    return result

In [16]:
def only_pos(dct):
    query = """
    SELECT token_id, sent_id
    FROM tokens
    WHERE pos = ?
    """
    cur.execute(query, (dct['pos'],))
    result = cur.fetchall()
    return result

In [17]:
def get_for_next(result, next):
    sentences = []
    if next['token']:
        item = 'token'
        if next['pos']:
            item += ' = ? AND pos'
    elif next['lemma']:
        item = 'lemma'
        if next['pos']:
            item += ' = ? AND pos'
    elif next['pos']:
        item = 'pos'
    t_query = f"""
    SELECT token_id, sent_id
    FROM tokens
    WHERE {item} = ? AND token_id = ? AND sent_id = ?
    """
    for token_id, sent_id in result:
        next_id = token_id + 1
        if ' ' in item:
            items = item.split(' = ? AND ')
            cur.execute(t_query, (next[items[0]], next[items[1]], next_id, sent_id))
        else:
            cur.execute(t_query, (next[item], next_id, sent_id))
        cur_sent = cur.fetchone()
        if cur_sent:
            sentences.append(cur_sent)
    return sentences

In [18]:
def sort_sentences(q_list):
    if q_list[0]['token'] and q_list[0]['pos']: # если в словарь токена записывать не пустые строки, а None
        result = token_and_pos(q_list[0]) # id токенов и предложений
    elif q_list[0]['lemma'] and q_list[0]['pos']:
        result = lemma_and_pos(q_list[0])
    elif q_list[0]['token'] and not q_list[0]['pos']:
        result = only_token(q_list[0])
    elif q_list[0]['lemma'] and not q_list[0]['pos']:
        result = only_lemma(q_list[0])
    else:
        result = only_pos(q_list[0])
    if result and len(q_list) > 1:
        sentences = get_for_next(result, q_list[1])
        if len(q_list) == 3:
            sentences = get_for_next(sentences, q_list[2])
            sentences = [obj[1] for obj in sentences]
            return sentences
        else:
            sentences = [obj[1] for obj in sentences]
            return sentences # получаем список айдишников предложений
    if result:
        result = [obj[1] for obj in result]
    return result

In [19]:
print(sort_sentences(q_list))

[1264, 1832, 2809, 2890, 4591, 5081, 8143]


In [20]:
ids = sort_sentences(q_list)

In [21]:
def get_sentences(ids):
    if len(ids) == 1:
        ids = f'({ids[0]})'
    else:
        ids = tuple(ids)
    posts_query = f'''
        SELECT poet, title, dirty_sent FROM sentences
        JOIN poems_to_info ON sentences.id_sent== poems_to_info.id_sent
        JOIN info ON poems_to_info.id_info == info.id_info
        WHERE sentences.id_sent in {ids}
    '''
    cur = conn.cursor()
    cur.execute(posts_query)
    result = cur.fetchall()
    return result

In [22]:
get_sentences(ids)

[('Владимир Маяковский', 'Гуляем', 'Раз шесть\nмоет лапкой\nна морде шерсть.'),
 ('Владимир Маяковский',
  'История Власа, лентяя и лоботряса',
  'Разошлись\nдругие\nв школы, —\nВлас\nу крана\nполуголый —\nне дремалось в школе чтоб,\nмоет нос\nи мочит лоб.'),
 ('Александр Блок',
  'На поле Куликовом',
  'Течет, грустит лениво\nИ моет берега.'),
 ('Сергей Есенин',
  'Не бродить, не мять в кустах багряных',
  'В тихий час, когда заря на крыше,\nКак котенок, моет лапкой рот,\nГовор кроткий о тебе я слышу\nВодяных поющих с ветром сот.'),
 ('Владимир Маяковский',
  'Про это',
  'Человек из-за 7-ми лет\nВолны устои стальные моют.'),
 ('Владимир Маяковский',
  'Про это',
  'Что хотите, буду делать даром —\nчистить,\nмыть,\nстеречь,\nмотаться,\nместь.'),
 ('Владимир Маяковский',
  'Что такое хорошо и что такое плохо',
  'Этот\nчистит валенки,\nмоет\nсам\nгалоши.')]